In [7]:
import os

from fractions import Fraction
from itertools import product
from qiskit_nature.second_q.properties import LatticeDrawStyle
from qiskit_nature.second_q.properties.lattices import Lattice, HyperCubicLattice
from qiskit_nature.second_q.hamiltonians.basic_operators import FermionicSpinor
from qiskit_nature.second_q.operators import FermionicOp, SpinOp,MixedOp
from qiskit_nature.second_q.mappers import QubitConverter, JordanWignerMapper, LogarithmicMapper, FermionicMapper

from qiskit_nature.second_q.hamiltonians.wilson_sun_hamiltonian import WilsonModel
import matplotlib.pyplot as plt
import numpy as np
import time

from qiskit_nature.second_q.properties.lattices import BoundaryCondition
from qiskit.algorithms import NumPyEigensolver
from qiskit.opflow import Zero
numpy_solver = NumPyEigensolver()
import json
from qiskit.opflow import PauliSumOp
from qiskit.quantum_info import SparsePauliOp
# reading the data from the file
def read_hamiltonian(name:str):
    path_to_file=f'/home/drudis/Documents/StoredOps/{name}.txt'
    with open(path_to_file) as f:
        data = f.read()
        
    # reconstructing the data as a dictionary
    js = json.loads(data)
    os.remove(path_to_file)

    all_pauli_ops = None
    for el in js['paulis']:
        pauOp = SparsePauliOp(el['label'][::-1]
        ,el['coeff']['real']+1.0j*el['coeff']['imag'])
        if all_pauli_ops is None:
            all_pauli_ops=pauOp
        else:
            all_pauli_ops+=pauOp
    return PauliSumOp(all_pauli_ops)

In [10]:
lattice_len = 3
a=1
r=2
m =3
spin =1
t = 2.4
e = 1.6
lam = 30  

some_lattice = HyperCubicLattice((lattice_len,),self_loops=False,boundary_condition=BoundaryCondition.OPEN)

# some_lattice.draw()


sigmax = np.array([[0.+0.j, 1.+0.j],
                   [1.+0.j, 0.+0.j]]).T

sigmay = np.array([[0.+0.j, 0.-1.j],
                   [0.+1.j, 0.+0.j]]).T

sigmaz = np.array([[1.+0.j,  0.+0.j],
                   [0.+0.j, -1.+0.j]]).T

dirac = [ sigmaz,sigmax*1j,sigmay*1j]


w_model = WilsonModel(  lattice = some_lattice,
                        lattice_constant=a,
                        wilson_parameter=r,
                        mass=m,
                        charge=e,#this is +-e charge or coupling
                        representation=dirac,#dirac_basis
                        flavours=1,
                        spin=spin,
                        electric_field=(0,0,0),
                        constraint_coefficient= lam
)
qubit_converter = QubitConverter(mappers = [JordanWignerMapper(),LogarithmicMapper()])

# hopping,mass,link_plaquette,gauge_regulator = w_model.mock_qubit_parts()
hamiltonian = qubit_converter.convert(w_model.second_q_ops())
hopp = qubit_converter.convert(w_model.hopping_term())
mass = qubit_converter.convert(MixedOp(([w_model.mass_term(),w_model.bosonic_qlm.idnty()],1.0)))
link = qubit_converter.convert(MixedOp(([w_model.fermionic_spinor.idnty(),w_model.link_term()],1.0)))
# np.testing.assert_allclose((hamiltonian - sum([hopp,mass,link])).to_matrix(),0,atol=1e-8)



regulator_term = []
for element in w_model.gauss_operators():
    converted_regulator = qubit_converter.convert(element)
    regulator_term.append(converted_regulator@converted_regulator)

regulator_term = sum(regulator_term)


#Here one needs to add a path to the old code and a version of python that can run it.
os.system(f"~/anaconda3/envs/aqua/bin/python /home/drudis/GH/old_lattice/export_hamiltonians.py {spin} {m} {t} {r} {a} {e} {lam}")
old_hamiltonian = read_hamiltonian("hamiltonian")
old_hopp = read_hamiltonian("hopp")
old_link = read_hamiltonian("link")
old_link = read_hamiltonian("mass")
old_regularizer = read_hamiltonian("regularizer")
qutip_ground_states = np.load('/home/drudis/Documents/StoredOps/ground_energies.npy')
os.remove('/home/drudis/Documents/StoredOps/ground_energies.npy')

0

Here we get the ground state energy for our hamiltonian. It is getting the right result without the gauge constraint, but not otherwise.

In [ ]:
print(numpy_solver.compute_eigenvalues(hamiltonian+lam*regulator_term).eigenvalues)
print(numpy_solver.compute_eigenvalues(old_hamiltonian).eigenvalues)
print("unregulated new method:", numpy_solver.compute_eigenvalues(hamiltonian).eigenvalues)
print(qutip_ground_states)


[230.4+3.47098762e-14j]
[-230.43161744+6.35443433e-07j]
unregulated new method: [-15.07582806+2.61013867e-17j]
[ -15.           -4.41814441    0.            0.          -15.50700126
 -230.43161744]
